# Generate training set fo DEM and Sat images

...for furhter use in pix2pix

Note: to extract and reproject DEM file for corresponding SAT image:

`rio warp eu_dem_v11_E40N20_3857.tif DEM_Aletsch_Winter.tif --like ../02_Sat/SAT_Aletsch_Winter.tif`

**Note**: set up to extract pairs of **150 x 200 pixels**, according to (current?) Sandbox DEMs




In [1]:
import numpy as np
import rasterio
import geopandas as gpd
from shapely.geometry import box
from rasterio.mask import mask
import progressbar

In [2]:
# Read the file with rasterio
# data_dem = rasterio.open("/Users/flow/Documents/01_work/16_Data/01_DEM/DEM_Aletsch_Winter.tif")
# data_sat = rasterio.open("/Users/flow/Documents/01_work/16_Data/02_Sat/SAT_Aletsch_Winter.tif")

# data_dem = rasterio.open("/Users/flow/Documents/01_work/16_Data/01_DEM/DEM_Bernese_Alps_summer.tif")
# data_sat = rasterio.open("/Users/flow/Documents/01_work/16_Data/02_Sat/SAT_Bernese_Alps_summer.tif")

# data_dem = rasterio.open("/Users/flow/Documents/01_work/16_Data/01_DEM/DEM_Alps_Summer.tif")
# data_sat = rasterio.open("/Users/flow/Documents/01_work/16_Data/02_Sat/SAT_Alps_Summer.tif")

data_dem = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/DEM_Alps_Summer.tif")
data_sat = rasterio.open("C:/Users/robin/Google Drive/BA/Wellmann SatImg/01b_DEM_SAT_pairs/SAT_Alps_Summer.tif")

# data_dem = rasterio.open("/Users/flow/Documents/01_work/16_Data/01_DEM/DEM_Alps_Summer_3.tif")
# data_sat = rasterio.open("/Users/flow/Documents/01_work/16_Data/02_Sat/SAT_Alps_Summer_3.tif")
                

In [ ]:
def generate_random_bbox(data_sat, nx=5000, ny=5000):
    """Generate random bbox within domain size of images
    
    nx = int: cells in x-direction
    ny = int: cells in y-direction
    """
    # set bounds from sat image for now:
    global_min_x = data_sat.bounds[0]
    global_min_y = data_sat.bounds[1]
    global_max_x = data_sat.bounds[2]
    global_max_y = data_sat.bounds[3]

    # get random location
    min_x = np.random.randint(global_min_x, global_max_x-nx)
    max_x = min_x + nx
    min_y = np.random.randint(global_min_y, global_max_y-ny)
    max_y = min_y + ny

    
    bbox = box(min_x, min_y, max_x, max_y)

    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0])
    
    return geo

def extract_images(data_sat, data_dem, geo):
    """Extract subimages from sat and dem
    """

    # Get the geometry coordinates
    coords = getFeatures(geo)

    # Clip the raster with the polygon
    out_img_sat, out_transform_sat = mask(data_sat, shapes=coords, crop=True)
    out_img_dem, out_transform_dtm = mask(data_dem, shapes=coords, crop=True)
    
    return out_img_sat, out_img_dem

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

Extract some more pixels, just in case (adjust then later in training itself):

In [ ]:
geo = generate_random_bbox(data_sat, nx=6200, ny=4700)

In [ ]:
out_img_sat, out_img_dem = extract_images(data_sat, data_dem, geo)

In [ ]:
# out_img_sat.shape, out_img_dem.shape

Remove edges (for some reason, edge effects on dem...) and Reorder axes for conventional shape format

In [ ]:
def reshape_and_reorder(out_img_sat, out_img_dem):
    # remove edges
    out_img_sat = out_img_sat[:,2:-2,2:-2]
    out_img_dem = out_img_dem[:,2:-2,2:-2]
    # reorder axes
    out_img_sat_reshape = np.moveaxis(out_img_sat, 0, -1)
    out_img_dem_reshape = np.moveaxis(out_img_dem, 0, -1)
    
    return out_img_sat_reshape, out_img_dem_reshape

np.save

In [ ]:
!pwd

## Loop to extract multiple image pairs

In [ ]:
n = 10

for i in progressbar.progressbar(range(10)):
    geo = generate_random_bbox(data_sat, nx=6200, ny=4700)
    out_img_sat, out_img_dem = extract_images(data_sat, data_dem, geo)
    out_img_sat_reshape, out_img_dem_reshape = reshape_and_reorder(out_img_sat, out_img_dem)
    # save in numpy binary
    np.save("../data/training_set/out_img_sat_%04d.npy" % i, out_img_sat_reshape)
    np.save("../data/training_set/out_img_dem_%04d.npy" % i, out_img_dem_reshape)
    

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(out_img_dem_reshape[:,:,0])